# Spark Analytics and Power BI Dashboard

## Price-Review Correlation Analysis in Cosmos DB

This notebook demonstrates how to analyze the relationship between product pricing and customer review ratings using the provided sample data. You'll discover insights about how price positioning within product categories correlates with customer satisfaction.

### 📋 Prerequisites & Setup Instructions

**Follow these steps in order:**

#### Step 1: Create Cosmos DB Container
1. **Create or use Cosmos DB artifact** in your Microsoft Fabric workspace:
    - Artifact name: `CosmosSampleDatabase` (or your preferred name)
    - This will be your database name used later
2. **Open Cosmos DB Data Explorer** for your artifact
3. **Create a new SampleData container**
    - Select SampleData from the **Home tab**

### Optionally: Manually create container
4. **Create a new container** with these settings:
   - Container name: `SampleData` (or your preferred name)
   - Partition key: `/categoryName`
5. **Upload sample data**: Use the **Upload** button in Data Explorer
6. **Select the file**: Navigate to `datasets/fabricSampleData.json` from this repository
7. **Verify upload**: Confirm all documents are loaded successfully

#### Step 2: Create Lakehouse and Shortcut
1. **Create a new Lakehouse** in your Fabric workspace
   - Give it a clear name like `cosmos_sample_lakehouse`
2. **Create a shortcut to your Cosmos DB**:
   - In your lakehouse, go to **Files** or **Tables** section
   - Click **New shortcut**
   - Select **Microsoft OneLake** → **Cosmos DB**
   - Choose your Cosmos DB artifact
   - Select the container you created in Step 1
3. **Note your names** for configuration:
   - Lakehouse name: `your_lakehouse_name`
   - Cosmos DB artifact/database: `CosmosSampleDatabase` (or what you named it)
   - Container/Table name: `SampleData` (or what you named it)

#### Step 3: Configure This Notebook
1. **Update cell 4** with your actual names:
   - `LAKEHOUSE_NAME` = your lakehouse name
   - `DATABASE_NAME` = your Cosmos database name  
   - `TABLE_NAME` = your container name

### ⚠️ IMPORTANT: Microsoft Fabric Environment Required
**This notebook MUST be run in Microsoft Fabric - it cannot be executed locally.**
- Requires Fabric **PySpark Python** runtime
- Uses Fabric-specific **Spark SQL** syntax for lakehouse data access
- Requires access to Cosmos DB data through lakehouse shortcuts
- Built-in authentication only works within Fabric notebooks

### 🎯 Learning Objectives
- **Analyze price-review correlations** across product categories
- **Understand category-relative pricing** vs. absolute price comparisons
- **Identify value opportunities** and pricing sweet spots
- **Generate business insights** from correlation patterns
- **Prepare data for Power BI** dashboards

### 📊 Analysis Approach
Instead of using arbitrary price bands (Budget/Premium), this analysis uses **category-relative positioning** to avoid comparing laptop prices to accessory prices. We calculate price percentiles within each category for meaningful insights.

### 🚨 Resource Naming Best Practice
**USING CONSISTENT NAMING**: This notebook uses `CosmosSampleDatabase` (no special characters) for the database name, which avoids SQL escaping issues. The helper function still handles names with dashes if you prefer a different naming convention.

In [ ]:
# Install required packages for data analysis and visualization
# This cell will only work in Microsoft Fabric notebook environment

print("📋 Installing packages for Microsoft Fabric notebook environment...")
%pip install pandas plotly

print("✅ Package installation complete!")
print("🚨 REMINDER: This notebook only works in Microsoft Fabric, not locally!")

In [ ]:
# Import required libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configuration: Update with YOUR Fabric lakehouse and database names
# These should match your actual Fabric setup after following the prerequisites

# Your lakehouse name in Fabric (where your mirrored data appears)
LAKEHOUSE_NAME = "cosmos_sample_lakehouse"  # Replace with your lakehouse name

# Your mirrored database name (from Cosmos DB mirroring)
DATABASE_NAME = "CosmosSampleDatabase"  # Replace with your mirrored database name

# Table name (should match your Cosmos DB container name)
TABLE_NAME = "SampleData"  # Usually matches your container name

# Helper function to properly escape database names with special characters
def escape_identifier(name):
    """Escape identifiers containing hyphens or special characters for SQL queries"""
    if '-' in name or ' ' in name or any(c in name for c in ['@', '$', '#']):
        return f"`{name}`"
    return name

# Create the properly formatted table reference for Spark SQL
ESCAPED_DATABASE = escape_identifier(DATABASE_NAME)
FULL_TABLE_NAME = f"{LAKEHOUSE_NAME}.{ESCAPED_DATABASE}.{TABLE_NAME}"

print("📍 Configuration Summary:")
print(f"  Lakehouse: {LAKEHOUSE_NAME}")
print(f"  Database: {DATABASE_NAME}")
print(f"  Table: {TABLE_NAME}")
print(f"  Full table reference: {FULL_TABLE_NAME}")
print("")
print("⚠️  UPDATE REQUIRED:")
print("Replace the values above with your actual Fabric lakehouse and database names")
print("💡 Database names with hyphens are automatically handled with backticks")

In [ ]:
# Connect to mirrored Cosmos DB data and validate dataset structure
print("🔄 Connecting to mirrored Cosmos DB data...")
print(f"📍 Table reference: {FULL_TABLE_NAME}")

try:
    # Test connection and load sample data
    test_query = f"SELECT * FROM {FULL_TABLE_NAME} LIMIT 10"
    df = spark.sql(test_query)
    
    # Get basic dataset information
    row_count_query = f"SELECT COUNT(*) as total FROM {FULL_TABLE_NAME}"
    row_count = spark.sql(row_count_query).collect()[0]['total']
    
    print(f"✅ Connected successfully!")
    print(f"📊 Total documents: {row_count:,}")
    print(f"📋 Columns: {df.columns}")
    
    # Validate we have the expected data structure for price-review analysis
    expected_columns = ['docType', 'currentPrice', 'stars', 'categoryName']
    missing_columns = [col for col in expected_columns if col not in df.columns]
    
    if missing_columns:
        print(f"⚠️ Missing expected columns: {missing_columns}")
        print("Please verify you're using the provided fabricSampleData.json")
    else:
        print("✅ Dataset structure validated for price-review analysis")
    
    # Show sample data
    print(f"\n📋 Sample data preview:")
    display(df)
    
    print("🎉 Ready to proceed with price-review correlation analysis!")
    
except Exception as e:
    print(f"❌ Connection error: {str(e)}")
    print("💡 Please check:")
    print("  • Lakehouse name matches your Fabric setup")
    print("  • Database name matches your mirrored Cosmos DB")
    print("  • Mirroring process has completed")
    print("  • You have access permissions to the lakehouse")

In [ ]:
# Load the complete dataset and prepare for price-review analysis
print("📊 Loading sample data for price-review correlation analysis...")

try:
    # Load the full dataset using Spark SQL
    df = spark.sql(f"SELECT * FROM {FULL_TABLE_NAME}")
    
    # Get row count for performance optimization decisions
    row_count = spark.sql(f"SELECT COUNT(*) as total FROM {FULL_TABLE_NAME}").collect()[0]['total']
    
    print(f"✅ Dataset loaded successfully!")
    print(f"📊 Total records: {row_count:,}")
    
    # Cache the dataframe for better performance in analysis queries
    df.cache()
    
    # Convert to Pandas for smaller datasets to enable certain visualizations
    if row_count < 100000:
        df_pandas = df.toPandas()
        print(f"✅ Converted to Pandas DataFrame for enhanced analysis")
    else:
        df_pandas = None
        print(f"📈 Large dataset - using Spark for optimal performance")
    
    # Register as temporary view for easy SQL queries
    df.createOrReplaceTempView("sample_data")
    print(f"✅ Registered as temporary view 'sample_data'")
    
    # Quick data validation for price-review analysis
    product_count = df.filter(df.docType == 'product').count()
    review_count = df.filter(df.docType == 'review').count()
    
    print(f"\n📋 Data Summary for Analysis:")
    print(f"  Products: {product_count:,}")
    print(f"  Reviews: {review_count:,}")
    print(f"  Categories: {df.select('categoryName').distinct().count()}")
    
    if product_count > 0 and review_count > 0:
        print(f"✅ Dataset ready for price-review correlation analysis!")
    else:
        print(f"⚠️ Missing required data - check your dataset structure")
    
except Exception as e:
    print(f"❌ Error loading data: {str(e)}")
    print("💡 Verify your configuration and lakehouse access")

In [ ]:
# Explore data structure and validate readiness for price-review correlation analysis
print("🔍 Exploring dataset structure for price-review analysis...")

try:
    # Document type distribution - should show products and reviews
    print("📋 Document Type Distribution:")
    doc_types = df.groupBy("docType").count().orderBy("count", ascending=False)
    display(doc_types.toPandas())
    
    # Category breakdown - shows product catalog diversity
    print("\n📈 Products and Reviews by Category:")
    category_breakdown = df.groupBy("categoryName", "docType").count().orderBy("categoryName", "docType")
    category_breakdown_pandas = category_breakdown.toPandas()
    
    # Pivot to show products vs reviews by category
    if df_pandas is not None:
        category_pivot = category_breakdown_pandas.pivot(index='categoryName', columns='docType', values='count').fillna(0)
        print("(Products and Reviews per Category)")
        display(category_pivot)
    else:
        display(category_breakdown_pandas)
    
    # Validate data completeness for correlation analysis
    print(f"\n📊 Data Validation for Correlation Analysis:")
    products_with_prices = df.filter((df.docType == 'product') & (df.currentPrice.isNotNull())).count()
    reviews_with_ratings = df.filter((df.docType == 'review') & (df.stars.isNotNull())).count()
    
    print(f"  Products with prices: {products_with_prices:,}")
    print(f"  Reviews with ratings: {reviews_with_ratings:,}")
    
    if products_with_prices > 0 and reviews_with_ratings > 0:
        print(f"✅ Dataset validated - ready for price-review correlation analysis!")
    else:
        print(f"⚠️ Missing price or rating data - check dataset completeness")
        
except Exception as e:
    print(f"❌ Error exploring data: {str(e)}")
    print("💡 Ensure data was loaded correctly in the previous step")

In [ ]:
# Price range analysis across categories using Spark SQL
print("? Price Analysis by Category:")

try:
    # Register the DataFrame as a temporary view for SQL queries
    df.createOrReplaceTempView("sample_data")
    
    # Price analysis query using Spark SQL
    price_analysis_query = """
    SELECT 
        categoryName,
        COUNT(DISTINCT id) as product_count,
        MIN(currentPrice) as min_price,
        AVG(currentPrice) as avg_price,
        MAX(currentPrice) as max_price,
        STDDEV(currentPrice) as price_std_dev
    FROM sample_data 
    WHERE docType = 'product' 
      AND currentPrice IS NOT NULL 
    GROUP BY categoryName 
    ORDER BY avg_price DESC
    """
    
    df_price_analysis = spark.sql(price_analysis_query)
    df_price_analysis_pandas = df_price_analysis.toPandas()
    
    display(df_price_analysis_pandas)
    
    # Create visualization of price ranges
    if df_price_analysis_pandas is not None and len(df_price_analysis_pandas) > 0:
        fig = px.bar(
            df_price_analysis_pandas,
            x='categoryName',
            y='avg_price',
            title="Average Price by Category",
            labels={'avg_price': 'Average Price ($)', 'categoryName': 'Category'},
            text='avg_price'
        )
        fig.update_traces(texttemplate='$%{text:,.0f}', textposition='outside')
        fig.update_xaxes(tickangle=45)
        fig.show()
        
        # Price range visualization
        fig2 = go.Figure()
        
        for _, row in df_price_analysis_pandas.iterrows():
            fig2.add_trace(go.Bar(
                name=row['categoryName'],
                x=[row['categoryName']],
                y=[row['max_price'] - row['min_price']],
                base=[row['min_price']],
                text=f"${row['min_price']:,.0f} - ${row['max_price']:,.0f}",
                textposition='inside'
            ))
        
        fig2.update_layout(
            title="Price Ranges by Category",
            xaxis_title="Category",
            yaxis_title="Price ($)",
            showlegend=False
        )
        fig2.update_xaxes(tickangle=45)
        fig2.show()
    
except Exception as e:
    print(f"❌ Error in price analysis: {str(e)}")
    print("💡 Make sure the data contains product records with currentPrice field")

In [ ]:
# Category-Relative Price Analysis using Spark SQL
# This analysis shows how ratings vary by price position WITHIN each category,
# avoiding the problem of comparing laptop prices to accessory prices.

print("? Category-Relative Price Position Analysis:")
print("This analysis shows how ratings vary by price position WITHIN each category,")
print("avoiding the problem of comparing laptop prices to accessory prices.")

try:
    price_position_query = """
    WITH ProductReviews AS (
        SELECT 
            p.currentPrice,
            r.stars,
            p.categoryName,
            -- Calculate relative price position within category (0-1 scale)
            (p.currentPrice - MIN(p.currentPrice) OVER (PARTITION BY p.categoryName)) / 
            NULLIF(MAX(p.currentPrice) OVER (PARTITION BY p.categoryName) - MIN(p.currentPrice) OVER (PARTITION BY p.categoryName), 0) as relative_price_position,
            -- Calculate price percentile within category
            PERCENT_RANK() OVER (PARTITION BY p.categoryName ORDER BY p.currentPrice) as price_percentile
        FROM sample_data p
        INNER JOIN sample_data r ON p.id = r.productId
        WHERE p.docType = 'product' 
          AND r.docType = 'review'
          AND p.currentPrice IS NOT NULL
          AND r.stars IS NOT NULL
    ),
    PricePositions AS (
        SELECT *,
            CASE 
                WHEN price_percentile < 0.25 THEN 'Bottom 25% (Lowest Price)'
                WHEN price_percentile < 0.50 THEN '25-50% (Below Average)'
                WHEN price_percentile < 0.75 THEN '50-75% (Above Average)'
                ELSE 'Top 25% (Highest Price)'
            END as price_position_category
        FROM ProductReviews
    )
    SELECT 
        categoryName,
        price_position_category,
        COUNT(*) as review_count,
        AVG(currentPrice) as avg_price,
        AVG(CAST(stars as DOUBLE)) as avg_rating,
        MIN(CAST(stars as DOUBLE)) as min_rating,
        MAX(CAST(stars as DOUBLE)) as max_rating,
        STDDEV(CAST(stars as DOUBLE)) as rating_std_dev,
        AVG(relative_price_position) as avg_relative_position,
        -- Rating distribution
        SUM(CASE WHEN stars = 5 THEN 1 ELSE 0 END) as five_star_count,
        SUM(CASE WHEN stars = 4 THEN 1 ELSE 0 END) as four_star_count,
        SUM(CASE WHEN stars <= 3 THEN 1 ELSE 0 END) as three_or_less_count
    FROM PricePositions
    GROUP BY categoryName, price_position_category
    HAVING COUNT(*) >= 3  -- Ensure statistical significance
    ORDER BY categoryName, avg_relative_position
    """
    
    df_price_position = spark.sql(price_position_query)
    df_price_position_pandas = df_price_position.toPandas()
    
    print("✅ Price position analysis complete!")
    display(df_price_position_pandas)
    
except Exception as e:
    print(f"❌ Error in price position analysis: {str(e)}")
    print("💡 Make sure the data was loaded correctly and temp view 'sample_data' exists")

In [ ]:
# Core Correlation Analysis: Price vs Review Stars by Category
print("🔍 Price-Review Correlation Analysis by Category:")

try:
    # Get raw price-review data for proper correlation calculation
    raw_correlation_query = """
    SELECT 
        p.categoryName,
        p.currentPrice,
        CAST(r.stars as DOUBLE) as rating
    FROM sample_data p
    INNER JOIN sample_data r ON p.id = r.productId
    WHERE p.docType = 'product' 
      AND r.docType = 'review'
      AND p.currentPrice IS NOT NULL
      AND r.stars IS NOT NULL
    """
    
    df_raw_correlation = spark.sql(raw_correlation_query)
    df_raw_correlation_pandas = df_raw_correlation.toPandas()
    
    print("✅ Raw correlation data loaded!")
    print(f"📊 Total price-review pairs: {len(df_raw_correlation_pandas):,}")
    
    # Calculate proper correlation coefficients by category using individual product-review pairs
    correlation_results = []
    
    for category in df_raw_correlation_pandas['categoryName'].unique():
        category_data = df_raw_correlation_pandas[df_raw_correlation_pandas['categoryName'] == category]
        
        if len(category_data) >= 10:  # Need sufficient data for meaningful correlation
            # Calculate Pearson correlation coefficient on individual data points
            correlation_coef = category_data['currentPrice'].corr(category_data['rating'])
            
            correlation_results.append({
                'categoryName': category,
                'total_reviews': len(category_data),
                'unique_products': category_data.groupby('currentPrice')['rating'].count().count(),
                'avg_price': category_data['currentPrice'].mean(),
                'avg_rating': category_data['rating'].mean(),
                'min_price': category_data['currentPrice'].min(),
                'max_price': category_data['currentPrice'].max(),
                'price_std': category_data['currentPrice'].std(),
                'rating_std': category_data['rating'].std(),
                'correlation_coefficient': correlation_coef
            })
    
    # Convert to DataFrame for analysis
    df_correlation_pandas = pd.DataFrame(correlation_results)
    
    print("✅ Correlation analysis complete!")
    display(df_correlation_pandas)
    
    # Enhanced correlation interpretation with proper thresholds
    print("\n📊 Correlation Analysis Results:")
    for _, row in df_correlation_pandas.iterrows():
        corr = row['correlation_coefficient']
        
        # Proper correlation strength interpretation matching data generation patterns
        if pd.isna(corr):
            strength = "No data"
            direction = ""
        elif abs(corr) >= 0.7:
            strength = "Strong"
            direction = "positive" if corr > 0 else "negative"
        elif abs(corr) >= 0.4:
            strength = "Moderate"
            direction = "positive" if corr > 0 else "negative"
        elif abs(corr) >= 0.2:
            strength = "Weak"
            direction = "positive" if corr > 0 else "negative"
        else:
            strength = "Very weak/None"
            direction = ""
        
        direction_text = f" ({direction})" if direction else ""
        print(f"  • {row['categoryName']}: {strength}{direction_text} correlation (r={corr:.3f})")
    
    # Visualization with proper correlation
    if len(df_correlation_pandas) > 0:
        fig = px.scatter(
            df_correlation_pandas,
            x='avg_price',
            y='avg_rating', 
            size='unique_products',
            color='correlation_coefficient',
            color_continuous_scale='RdBu',
            color_continuous_midpoint=0,
            title='Price vs Rating Correlation by Category',
            labels={
                'avg_price': 'Average Price ($)', 
                'avg_rating': 'Average Rating (Stars)',
                'correlation_coefficient': 'Correlation Coefficient'
            },
            hover_data=['total_reviews', 'correlation_coefficient']
        )
        fig.show()
        
        # Correlation strength distribution
        df_correlation_pandas['correlation_strength'] = df_correlation_pandas['correlation_coefficient'].apply(
            lambda x: 'Strong (|r|≥0.7)' if abs(x) >= 0.7 else
                     'Moderate (|r|≥0.4)' if abs(x) >= 0.4 else
                     'Weak (|r|≥0.2)' if abs(x) >= 0.2 else
                     'Very weak/None (|r|<0.2)'
        )
        
        strength_counts = df_correlation_pandas['correlation_strength'].value_counts()
        fig2 = px.pie(
            values=strength_counts.values,
            names=strength_counts.index,
            title="Distribution of Correlation Strengths Across Categories"
        )
        fig2.show()
    
except Exception as e:
    print(f"❌ Error in correlation analysis: {str(e)}")
    print("💡 Make sure the data contains both product and review records with proper relationships")

In [ ]:
# Detailed Product Analysis: Best and Worst Performers
# Identify products that over/under-perform relative to their price point

print("🏆 Product Performance Analysis:")

try:
    detailed_analysis_query = """
    WITH ProductStats AS (
        SELECT 
            p.id as product_id,
            p.name as product_name,
            p.categoryName,
            p.currentPrice,
            COUNT(r.id) as review_count,
            AVG(CAST(r.stars as DOUBLE)) as avg_rating,
            STDDEV(CAST(r.stars as DOUBLE)) as rating_consistency
        FROM sample_data p
        INNER JOIN sample_data r ON p.id = r.productId
        WHERE p.docType = 'product' 
          AND r.docType = 'review'
          AND p.currentPrice IS NOT NULL
          AND r.stars IS NOT NULL
        GROUP BY p.id, p.name, p.categoryName, p.currentPrice
        HAVING COUNT(r.id) >= 2  -- Products with at least 2 reviews
    ),
    CategoryAverages AS (
        SELECT 
            categoryName,
            AVG(currentPrice) as category_avg_price,
            AVG(avg_rating) as category_avg_rating
        FROM ProductStats
        GROUP BY categoryName
    )
    SELECT 
        ps.product_name,
        ps.categoryName,
        ps.currentPrice,
        ps.avg_rating,
        ps.review_count,
        ca.category_avg_price,
        ca.category_avg_rating,
        -- Performance metrics relative to category
        (ps.currentPrice - ca.category_avg_price) as price_vs_category,
        (ps.avg_rating - ca.category_avg_rating) as rating_vs_category,
        -- Value score: High rating, reasonable price relative to category
        CASE 
            WHEN ps.avg_rating > ca.category_avg_rating AND ps.currentPrice <= ca.category_avg_price THEN 'Great Value'
            WHEN ps.avg_rating > ca.category_avg_rating AND ps.currentPrice > ca.category_avg_price THEN 'Premium Quality'
            WHEN ps.avg_rating <= ca.category_avg_rating AND ps.currentPrice <= ca.category_avg_price THEN 'Budget Option'
            ELSE 'Poor Value'
        END as value_category
    FROM ProductStats ps
    JOIN CategoryAverages ca ON ps.categoryName = ca.categoryName
    ORDER BY ps.avg_rating DESC, ps.currentPrice ASC
    """
    
    df_detailed = spark.sql(detailed_analysis_query)
    df_detailed_pandas = df_detailed.toPandas()
    
    print("✅ Product performance analysis complete!")
    
    # Show top 15 products
    print("\n🏆 Top 15 Products by Rating:")
    display(df_detailed_pandas.head(15))
    
    # Value category distribution
    if len(df_detailed_pandas) > 0:
        value_distribution = df_detailed_pandas['value_category'].value_counts()
        print(f"\n📈 Value Category Distribution:")
        for category, count in value_distribution.items():
            print(f"  {category}: {count} products")
    
except Exception as e:
    print(f"❌ Error in detailed product analysis: {str(e)}")
    print("💡 Make sure the data was loaded correctly and temp view 'sample_data' exists")

In [ ]:
# Create comprehensive visualization dashboard
# Multiple charts showing correlation patterns and category-relative analysis

print("📊 Creating comprehensive visualization dashboard...")

# Ensure we have pandas DataFrames for visualization
try:
    # Convert Spark DataFrames to Pandas if needed
    if not isinstance(df_correlation_pandas, pd.DataFrame):
        print("Converting correlation data to Pandas DataFrame...")
        df_correlation_pandas = df_correlation.toPandas()
    
    if not isinstance(df_price_position_pandas, pd.DataFrame):
        print("Converting price position data to Pandas DataFrame...")
        df_price_position_pandas = df_price_position.toPandas()
    
    if not isinstance(df_detailed_pandas, pd.DataFrame):
        print("Converting detailed analysis data to Pandas DataFrame...")
        df_detailed_pandas = df_detailed.toPandas()
    
    print("✅ Data conversion complete!")
    
except Exception as e:
    print(f"❌ Error converting data: {str(e)}")
    print("💡 Make sure all previous analysis steps completed successfully")

# Set up the subplot layout
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Price vs Rating by Category',
        'Price Position vs Rating (Category-Relative)', 
        'Category Price Distribution',
        'Value Categories Distribution'
    ],
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"type": "pie"}]]
)

# 1. Scatter plot: Price vs Rating by Category
for category in df_correlation_pandas['categoryName'].unique():
    category_data = df_correlation_pandas[df_correlation_pandas['categoryName'] == category]
    fig.add_trace(
        go.Scatter(
            x=category_data['avg_price'],
            y=category_data['avg_rating'],
            mode='markers',
            marker=dict(size=category_data['unique_products'], sizemode='diameter', sizeref=2),
            name=category,
            text=category_data['categoryName'],
            hovertemplate='<b>%{text}</b><br>Price: $%{x:,.0f}<br>Rating: %{y:.2f}<br>Products: %{marker.size}<extra></extra>'
        ),
        row=1, col=1
    )

# 2. Category-Relative Price Position Analysis
# Aggregate price position data for visualization
position_ratings = df_price_position_pandas.groupby('price_position_category').agg({
    'avg_rating': 'mean',
    'review_count': 'sum',
    'avg_relative_position': 'mean'
}).reset_index()

fig.add_trace(
    go.Bar(
        x=position_ratings['price_position_category'],
        y=position_ratings['avg_rating'],
        name='Avg Rating by Price Position',
        showlegend=False,
        text=[f'{rating:.2f}' for rating in position_ratings['avg_rating']],
        textposition='auto',
        hovertemplate='<b>%{x}</b><br>Avg Rating: %{y:.2f}<br>Total Reviews: %{customdata}<extra></extra>',
        customdata=position_ratings['review_count']
    ),
    row=1, col=2
)

# 3. Box plot: Price distribution by category
fig.add_trace(
    go.Box(
        y=df_correlation_pandas['avg_price'],
        x=df_correlation_pandas['categoryName'],
        name='Price Distribution',
        showlegend=False,
        hovertemplate='<b>%{x}</b><br>Price: $%{y:,.0f}<extra></extra>'
    ),
    row=2, col=1
)

# 4. Pie chart: Value categories
value_counts = df_detailed_pandas['value_category'].value_counts()
fig.add_trace(
    go.Pie(
        labels=value_counts.index,
        values=value_counts.values,
        name="Value Categories",
        hovertemplate='<b>%{label}</b><br>Count: %{value}<br>%{percent}<extra></extra>'
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="Product Price-Rating Correlation Analysis Dashboard",
    showlegend=True
)

# Update axes labels
fig.update_xaxes(title_text="Average Price ($)", row=1, col=1)
fig.update_yaxes(title_text="Average Rating", row=1, col=1)
fig.update_xaxes(title_text="Price Position Within Category", row=1, col=2)
fig.update_yaxes(title_text="Average Rating", row=1, col=2)
fig.update_xaxes(title_text="Category", row=2, col=1)
fig.update_yaxes(title_text="Price ($)", row=2, col=1)

fig.show()

print("\n🎯 Key Insights from Category-Relative Analysis:")
print("1. Price positions are calculated WITHIN each category, making comparisons meaningful")
print("2. Look for patterns where higher price positions correlate with higher ratings")
print("3. Value categories identify products that over/under-perform relative to category pricing")
print("4. This approach avoids the problem of comparing laptop prices to accessory prices")

In [ ]:
# Generate actionable business insights from price-review correlation analysis
print("🎯 BUSINESS INSIGHTS FROM PRICE-REVIEW CORRELATION ANALYSIS")
print("="*65)

try:
    # 1. Category performance insights
    best_correlation = df_correlation_pandas.nlargest(3, 'avg_rating')
    print("\n📈 TOP PERFORMING CATEGORIES (by average rating):")
    for _, row in best_correlation.iterrows():
        print(f"  • {row['categoryName']}: {row['avg_rating']:.2f} stars")
        print(f"    Average price: ${row['avg_price']:,.0f}")

    # 2. Value opportunity identification
    if 'df_detailed_pandas' in locals() and len(df_detailed_pandas) > 0:
        great_value_products = df_detailed_pandas[df_detailed_pandas['value_category'] == 'Great Value']
        print(f"\n💰 VALUE OPPORTUNITIES:")
        print(f"  • {len(great_value_products)} products identified as 'Great Value'")
        print(f"  • These products rate above average while priced at/below category average")
        
        if len(great_value_products) > 0:
            print(f"  • Average price: ${great_value_products['currentPrice'].mean():,.0f}")
            print(f"  • Average rating: {great_value_products['avg_rating'].mean():.2f} stars")

    # 3. Category-relative pricing insights
    if 'df_price_position_pandas' in locals() and len(df_price_position_pandas) > 0:
        position_summary = df_price_position_pandas.groupby('price_position_category')['avg_rating'].mean().sort_values(ascending=False)
        best_position = position_summary.index[0]
        print(f"\n🏆 OPTIMAL PRICE POSITIONING:")
        print(f"  • {best_position}: {position_summary.iloc[0]:.2f} avg rating")
        print(f"  • This shows the optimal price tier within each category")

    # 4. Strategic recommendations based on correlation patterns
    print(f"\n🎯 STRATEGIC RECOMMENDATIONS:")
    print(f"  1. Focus on categories with strong price-quality correlation")
    print(f"  2. Investigate 'Poor Value' products for pricing optimization")
    print(f"  3. Use 'Great Value' products for competitive positioning")
    print(f"  4. Position products in optimal price percentiles within categories")
    print(f"  5. Monitor rating trends to predict pricing pressure")

    # 5. Category-specific insights
    print(f"\n📊 CATEGORY-SPECIFIC INSIGHTS:")
    for _, row in df_correlation_pandas.iterrows():
        correlation_coef = row.get('correlation_coefficient', 0)
        
        # Use the same correlation interpretation as the main analysis
        if abs(correlation_coef) >= 0.7:
            correlation_strength = "Strong"
        elif abs(correlation_coef) >= 0.4:
            correlation_strength = "Moderate"  
        elif abs(correlation_coef) >= 0.2:
            correlation_strength = "Weak"
        else:
            correlation_strength = "Very weak/None"
            
        direction = "positive" if correlation_coef > 0 else "negative" if correlation_coef < 0 else ""
        direction_text = f" ({direction})" if direction and abs(correlation_coef) >= 0.2 else ""
        
        print(f"  • {row['categoryName']}: {correlation_strength}{direction_text} correlation (r={correlation_coef:.3f})")

    print(f"\n✅ Price-review correlation analysis complete!")
    print(f"📊 Use these insights to optimize pricing strategy and product positioning")
    
except Exception as e:
    print(f"❌ Error generating insights: {str(e)}")
    print("💡 Ensure all analysis steps completed successfully")